---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [5]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [7]:
import datetime as dt
import numpy as np
def date_sorter():
    global df
    df=df.str.replace(r'January', 'Jan')
    df=df.str.replace(r'Janaury', 'Jan')
    df=df.str.replace(r'February', 'Feb')
    df=df.str.replace(r'March', 'Mar')
    df=df.str.replace(r'April', 'Apr')
    df=df.str.replace(r'June', 'Jun')
    df=df.str.replace(r'July', 'Jul')
    df=df.str.replace(r'August', 'Aug')
    df=df.str.replace(r'September', 'Sep')
    df=df.str.replace(r'October', 'Oct')
    df=df.str.replace(r'November', 'Nov')
    df=df.str.replace(r'December', 'Dec')
    df=df.str.replace(r'Decemeber', 'Dec')
    #extract date
    df1 = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2,4}))')
    left = ~df.index.isin([x[0] for x in df1.index])
    df1 = df1.append(df[left].str.extractall(r'(?P<origin>(?P<day>([0-2]?[0-9])|([3][01])) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).?-? (?P<year>\d?\d?\d\d))'))
    left = ~df.index.isin([x[0] for x in df1.index])
    df1 = df1.append(df[left].str.extractall(r'(?P<origin>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).?-? (?P<day>\d?\d)?,? (?P<year>\d?\d?\d\d))'))
    left = ~df.index.isin([x[0] for x in df1.index])
    df1 = df1.append(df[left].str.extractall(r'(?P<origin>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).?-? ?(?P<year>\d?\d?\d\d))'))
    left = ~df.index.isin([x[0] for x in df1.index])
    df1 = df1.append(df[left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/](?P<year>\d?\d?\d\d))'))
    left = ~df.index.isin([x[0] for x in df1.index])
    df1 = df1.append(df[left].str.extractall(r'(?P<origin>(?P<year>19\d\d))'))
    left = ~df.index.isin([x[0] for x in df1.index])
    df1 = df1.append(df[left].str.extractall(r'(?P<origin>(?P<year>20\d\d))'))
    left = ~df.index.isin([x[0] for x in df1.index])

    df1['year']=df1['year'].apply(lambda x: '19'+x if len(x)==2 else x)
    df1['month']=df1['month'].apply(lambda x: 1 if x=='Jan' else x)
    df1['month']=df1['month'].apply(lambda x: 1 if x=='01' else x)
    df1['month']=df1['month'].apply(lambda x: 2 if x=='Feb' else x)
    df1['month']=df1['month'].apply(lambda x: 2 if x=='02' else x)
    df1['month']=df1['month'].apply(lambda x: 3 if x=='Mar' else x)
    df1['month']=df1['month'].apply(lambda x: 3 if x=='03' else x)
    df1['month']=df1['month'].apply(lambda x: 4 if x=='Apr' else x)
    df1['month']=df1['month'].apply(lambda x: 4 if x=='04' else x)
    df1['month']=df1['month'].apply(lambda x: 5 if x=='May' else x)
    df1['month']=df1['month'].apply(lambda x: 5 if x=='05' else x)
    df1['month']=df1['month'].apply(lambda x: 6 if x=='Jun' else x)
    df1['month']=df1['month'].apply(lambda x: 6 if x=='06' else x)
    df1['month']=df1['month'].apply(lambda x: 7 if x=='Jul' else x)
    df1['month']=df1['month'].apply(lambda x: 7 if x=='07' else x)
    df1['month']=df1['month'].apply(lambda x: 8 if x=='Aug' else x)
    df1['month']=df1['month'].apply(lambda x: 8 if x=='08' else x)
    df1['month']=df1['month'].apply(lambda x: 9 if x=='Sep' else x)
    df1['month']=df1['month'].apply(lambda x: 9 if x=='09' else x)
    df1['month']=df1['month'].apply(lambda x: 10 if x=='Oct' else x)
    df1['month']=df1['month'].apply(lambda x: 11 if x=='Nov' else x)
    df1['month']=df1['month'].apply(lambda x: 12 if x=='Dec' else x)
    df1=pd.DataFrame(data=df1)
    df1=df1.fillna(1)
    df1['month']=df1['month'].apply(lambda x: str(x))
    df1['day']=df1['day'].apply(lambda x: str(x))
    df1['year']=df1['year'].apply(lambda x: str(x))
    
    df1['date'] = df1['month'] + '/' + df1['day'] + '/' + df1['year']
    df1['date'] = pd.to_datetime(df1['date'])
    df1.sort_values(by='date', inplace=True)
    df2 = pd.Series(list(df1.index.labels[0]))
    return df2
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    244
477    480
478    286
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64